In [1]:
import pandas as pd

In [2]:
from dgutils.pandas import add_column

/Users/alicegao/anaconda2/envs/yeast_d_cell/lib/python3.7/site-packages/genome_kit/data_manager.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
import matplotlib.pyplot as plt 
import seaborn as sns
import pandas as pd
sns.set(color_codes=True)
import cufflinks as cf
cf.go_offline()
cf.set_config_file(theme='ggplot')

/Users/alicegao/anaconda2/envs/yeast_d_cell/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/alicegao/anaconda2/envs/yeast_d_cell/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/alicegao/anaconda2/envs/yeast_d_cell/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [4]:
from scipy.stats import pearsonr

In [5]:
df_pred = pd.read_csv('data/for_comparison/costonzo_2009_pvalue005_prediction_growth', sep='\t',
                     header=None, names=['g1', 'g2', 'pred'])

In [6]:
# ExE - no pred pass 0.05 curoff?
# df_exe = pd.read_csv('data/for_comparison/SGA_ExE.txt', sep='\t')[['Query Strain ID', 'Array Strain ID', 'Double mutant fitness']]

# ExN ~500
df_raw = pd.read_csv('data/for_comparison/SGA_ExN_NxE.txt', sep='\t')[['Query Strain ID', 'Array Strain ID', 'Double mutant fitness']]


# # NxN 398664 (majority)
# df_raw = pd.read_csv('data/for_comparison/SGA_NxN.txt', sep='\t')[['Query Strain ID', 'Array Strain ID', 'Double mutant fitness']]


In [7]:
# process raw data to extract gene id

def _get_gene_id(strain_id):
    gene_id, _ = strain_id.split('_')
    return gene_id

df_raw = add_column(df_raw, 'g1', ['Query Strain ID'], _get_gene_id)
df_raw = add_column(df_raw, 'g2', ['Array Strain ID'], _get_gene_id)
df_raw.drop(columns=['Query Strain ID', 'Array Strain ID'], inplace=True)
df_raw.rename(columns={'Double mutant fitness': 'fitness'}, inplace=True)

In [8]:
# make gene id pair a sorted tuple, for joining
df_pred = add_column(df_pred, 'gene_pair', ['g1', 'g2'], lambda x, y: tuple(sorted([x, y])))
df_raw = add_column(df_raw, 'gene_pair', ['g1', 'g2'], lambda x, y: tuple(sorted([x, y])))
df_pred.drop(columns=['g1', 'g2'], inplace=True)
df_raw.drop(columns=['g1', 'g2'], inplace=True)

In [9]:
# combine duplicates
# take median for now
df_raw = df_raw.groupby(by='gene_pair', as_index=False).agg('median')

In [10]:
# intersection of the two data 
# (since we didn't load everything: raw is only a subset, pred is p<0.05 only)
print(len(df_raw), len(df_pred))
df = pd.merge(df_raw, df_pred, on='gene_pair', how='inner')
print(len(df))

3361604 413400
504


In [11]:
df.head()

,gene_pair,fitness,pred
0,"(YAL010C, YGR092W)",0.55095,1.073540
1,"(YAL013W, YGR092W)",0.66500,1.019644
2,"(YAL024C, YHR191C)",0.70615,0.814487
3,"(YAR003W, YGR092W)",0.65865,0.649298
4,"(YAR035W, YHR191C)",0.82000,0.887804


In [12]:
pearsonr(df['fitness'], df['pred'])

(0.03919683024583753, 0.37987903689300107)

In [13]:
# do not plot if we have too many data points!
if len(df) < 10000:
    df.iplot(kind='scatter', mode='markers',
            x='fitness', y='pred', size=2)